In [3]:
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
import glob
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, LSTM, BatchNormalization, GlobalAveragePooling1D
from sklearn.metrics import classification_report
from scipy.stats import skew, kurtosis
import requests

In [4]:
API_BASE_URL = "https://bills-api.parliament.uk/api/v1/Bills"

In [5]:

def get_bill_info(bill_id):
    url = f"{API_BASE_URL}/{bill_id}"
    headers = {"accept": "application/json"}
    response = requests.get(url, headers=headers)
    try:
        data = response.json()
        if isinstance (data, dict):
            long_title = data.get("longTitle", "N/A")
            short_title = data.get("shortTitle", "N/A")
            orig_house = data.get("originatingHouse", "N/A")
            status = check_bill_progress(bill_id)
            sessionID = data.get("introducedSessionId", "N/A")
           
            
            return bill_id, long_title, short_title, orig_house, status, sessionID
    except requests.exceptions.JSONDecodeError:
        print(f"Error: Response for Bill ID {bill_id} is not valid JSON. Response:", response.text)
        return False
    
    

        




In [6]:

def count_bills_in_session(session_id):
    """Fetch the number of bills that contain a specific session ID."""
    params = {
        "Session": session_id,
        "Take": 1  # Only request one item to check total count
    }
    
    headers = {"accept": "application/json"}
    response = requests.get(API_BASE_URL, headers=headers, params=params)

    try:
        data = response.json()
        if isinstance(data, dict) and "totalResults" in data:
            return data["totalResults"]  # Total number of bills with this session ID
        else:
            print("Unexpected API response format:", data)
            return 0
    except requests.exceptions.JSONDecodeError:
        print("Error: Response is not valid JSON. Response:", response.text)
        return 0
    
print (count_bills_in_session(37))

355


In [7]:
def check_bill_progress(bill_id):
    """Checks if a bill is an act using the isAct field from the API."""
    url = f"https://bills-api.parliament.uk/api/v1/Bills/{bill_id}"  
    headers = {"accept": "application/json"}  
    
    response = requests.get(url, headers=headers)

    try:
        data = response.json()  
        if 'isAct' in data:
            return 2 if data['isAct'] else 1
        else:
            print(f"'isAct' field not found for Bill ID {bill_id}")
            return 3  

    except requests.exceptions.JSONDecodeError:
        print(f"Error: Response for Bill ID {bill_id} is not valid JSON. Response:", response.text)
        return False
print (
check_bill_progress(799))

2


In [8]:
def populate_dataframes(bills, f_commons, f_lords, s_commons, s_lords, weird):
    columns1 = ["Bill Id", "Long Title", "Short Title", "Origin_House", "Status", "SessionID"]

    rows_f_commons, rows_f_lords, rows_s_commons, rows_s_lords, rows_weird = [], [], [], [], []
    
    for bill in bills:
        bill_id = bill.get("billId", "N/A")
        bill_id, long_title, short_title, orig_house, status, sessionID = get_bill_info(bill_id)
        
        new_row = [bill_id, long_title, short_title, orig_house, status, sessionID]
        
        if status == 1:
            if orig_house == "Commons":
                rows_f_commons.append(new_row)
            elif orig_house == "Lords":
                rows_f_lords.append(new_row)
        elif status == 2:
            if orig_house == "Commons":
                rows_s_commons.append(new_row)
            elif orig_house == "Lords":
                rows_s_lords.append(new_row)
        elif status == 3:
            rows_weird.append(new_row)
    

    f_commons = pd.concat([pd.DataFrame(rows_f_commons, columns=columns1), f_commons], ignore_index=True)
    f_lords = pd.concat([pd.DataFrame(rows_f_lords, columns=columns1), f_lords], ignore_index=True)
    s_commons = pd.concat([pd.DataFrame(rows_s_commons, columns=columns1), s_commons], ignore_index=True)
    s_lords = pd.concat([pd.DataFrame(rows_s_lords, columns=columns1), s_lords], ignore_index=True)
    weird = pd.concat([pd.DataFrame(rows_weird, columns=columns1), weird], ignore_index=True)
    
    return f_commons, f_lords, s_commons, s_lords, weird


In [ ]:
import datetime




#last session 39
columns = ["Bill Id", "Long Title", "Short Title", "Origin_House", "Status", "SessionID"]

S_commons0 = pd.DataFrame(columns=columns)
F_commons0 = pd.DataFrame(columns=columns)
S_lords0 = pd.DataFrame(columns=columns)
F_lords0 = pd.DataFrame(columns=columns)
weirdList0 = pd.DataFrame(columns=columns)



def get_bills_by_session(session_id, num_bills):
    """Fetch bills from a specific session ID."""
    params = {
        "Session": session_id,
        "Take": num_bills  
    }
    
    headers = {"accept": "application/json"}
    response = requests.get(API_BASE_URL, headers=headers, params=params)

    try:
        data = response.json()
        if isinstance(data, dict) and "items" in data:
            return data["items"]  
        else:
            print("Unexpected API response format:", data)
            return []
    except requests.exceptions.JSONDecodeError:
        print("Error: Response is not valid JSON. Response:", response.text)
        return []
    



session_id = int(input("Enter Session ID: "))
#num_bills = int(input("Enter number of bills to retrieve: "))






start_session = session_id
end_session = 39 

#num_bills0= count_bills_in_session(session_id)
#bills0= get_bills_by_session(session_id, num_bills)


now = datetime.datetime.now()

# Print just the time part
print("Code starts executing at :", now.strftime("%H:%M:%S"))  
    

    
    
for session in range(start_session, end_session + 1):
    now = datetime.datetime.now()
    print(f"counting bills in session {session} starts at :", now.strftime("%H:%M:%S"))  
    num_bills = count_bills_in_session(session)
    now = datetime.datetime.now()
    print(f"counting bills in session {session} finish at :", now.strftime("%H:%M:%S"))  
    bills0 = get_bills_by_session(session, num_bills)
    now = datetime.datetime.now()
    print(f"Bills0 computed for session {session} finish at :", now.strftime("%H:%M:%S"))  
    
    F_commons0 , F_lords0, S_commons0, S_lords0, weirdList0 = populate_dataframes(bills0, F_commons0, F_lords0, S_commons0, S_lords0, weirdList0)    
    
    now = datetime.datetime.now()
    print(f"Dataframes appended at {session} finish at :", now.strftime("%H:%M:%S"))
    



display(F_commons0)
display(F_lords0)
display(S_commons0)
display(S_lords0)


#print ("This is the weird list")
#display(weirdList0)

F_commons0.to_csv(   f"18_{start_session} to {end_session}.csv"   , index=False)
F_lords0.to_csv(     f"19_{start_session} to {end_session}.csv"   , index = False )
S_lords0.to_csv(     f"29_{start_session} to {end_session}.csv", index = False )
S_commons0.to_csv(   f"28_{start_session} to {end_session}.csv", index= False)
weirdList0.to_csv(   f"errors {start_session} to {end_session}.csv", index= False )



print("Code finished executing ")
# Get the current datetime
now2 = datetime.datetime.now()

# Print just the time part
print("Time Code Finished Executing:", now2.strftime("%H:%M:%S"))




In [9]:

total_bills = 0
yearsWithBills=0

for i in range (0,50):
    bills = count_bills_in_session(i)
    if bills != 0:
        yearsWithBills = yearsWithBills + 1
    print( (f"Bills in session {i} is {bills}"))
    total_bills = total_bills + bills
print(total_bills)
print (f"Years with bills {yearsWithBills}")



Bills in session 0 is 0
Bills in session 1 is 0
Bills in session 2 is 0
Bills in session 3 is 0
Bills in session 4 is 0
Bills in session 5 is 0
Bills in session 6 is 0
Bills in session 7 is 0
Bills in session 8 is 0
Bills in session 9 is 0
Bills in session 10 is 0
Bills in session 11 is 0
Bills in session 12 is 0
Bills in session 13 is 0
Bills in session 14 is 0
Bills in session 15 is 0
Bills in session 16 is 0
Bills in session 17 is 0
Bills in session 18 is 2
Bills in session 19 is 5
Bills in session 20 is 150
Bills in session 21 is 160
Bills in session 22 is 163
Bills in session 23 is 113
Bills in session 24 is 313
Bills in session 25 is 175
Bills in session 26 is 213
Bills in session 27 is 195
Bills in session 28 is 193
Bills in session 29 is 198
Bills in session 30 is 459
Bills in session 31 is 0
Bills in session 32 is 0
Bills in session 33 is 0
Bills in session 34 is 57
Bills in session 35 is 334
Bills in session 36 is 302
Bills in session 37 is 355
Bills in session 38 is 235
Bill